In [6]:
import matplotlib.pyplot as plt
from scipy.io import wavfile
import argparse
import os
from glob import glob
import numpy as np
import pandas as pd
from librosa.core import resample, to_mono
from tqdm import tqdm


Se leen los archivos .wav de las carpetas 'Archivos_wav' y 'Archivos_wav_test'. Se realiza un downsampling a 16 Khz, y se divide cada muestra a fin de obtener muestras de 2 segundos. Las muestras procesadas se guardan en 'Archivos_procesados' y 'Archivos_procesados_test'.

In [7]:
def downsampling(path, sr):
    rate, wav = wavfile.read(path)
    wav = wav.astype(np.float32, order='F')
    try:
        tmp = wav.shape[1]
        wav = to_mono(wav.T)
    except:
        pass
    wav = resample(wav, rate, sr)
    wav = wav.astype(np.int16)
    return sr, wav

In [8]:
def save_sample(sample, rate, target_dir, fn, ix):
    fn = fn.split('.wav')[0]
    dst_path = os.path.join(target_dir.split('.')[0], fn+'_{}.wav'.format(str(ix)))
    if os.path.exists(dst_path):
        return
    wavfile.write(dst_path, rate, sample)

In [12]:
src_root = 'Archivos_wav'
src_root_test = 'Archivos_wav_test'
dst_root = 'Archivos_procesados'
dst_root_test = 'Archivos_procesados_test'
dt = 2

wav_paths = glob('{}/**'.format(src_root), recursive=True)
wav_paths = [x for x in wav_paths if '.wav' in x]

wav_paths_test = glob('{}/**'.format(src_root_test), recursive=True)
wav_paths_test = [x for x in wav_paths_test if '.wav' in x]


dirs = os.listdir(src_root)
dirs_test = os.listdir(src_root)

os.mkdir(dst_root)
os.mkdir(dst_root_test)

clases = os.listdir(src_root)

In [13]:
for i in clases:  #_cls en este caso va a ser primero 'Charo' despues 'Gonza'
        target_dir = os.path.join(dst_root, i)    #Esta juntando el directorio de destino que llamamos 'Archivos_procesados' con el directorio de la clase
        target_dir_test = os.path.join(dst_root_test, i)
        os.mkdir(target_dir)  #Si el directorio no existe lo crea
        os.mkdir(target_dir_test)
        src_dir = os.path.join(src_root, i) #En este tiene el directorio Archivos_wav/(Charo o Gonza)
        src_dir_test = os.path.join(src_root_test, i)
        for fn in tqdm(os.listdir(src_dir)):   #Aca loopea sobre cada archivo dentro de Archivos_wav//('Charo', 'Gonza'): Es decir primero Charo1.wav, Charo2.wav
            src_fn = os.path.join(src_dir, fn) #Aca obtiene el directorio total del archivo que va a procesar
            rate, wav = downsampling(src_fn, args.sr)  #Aca llama a la función downsample_mono. Le pasa Archivos_wav/Charo/el wav y el sr que se quiere (16000)
            #mask, y_mean = envelope(wav, rate, threshold=args.threshold)  #Por ahora la evito
            #wav = wav[mask]
            delta_sample = int(dt*rate)   # 16000 * 2 = 32000 : Cada sample va a tener 32000 muestras. Las que queden menores se rellenan con 0.

            # cleaned audio is less than a single sample
            # pad with zeros to delta_sample size
            if wav.shape[0] < delta_sample: #Si el archivo es mas corto que los 2 segundos que queremos le agregamos ceros
                sample = np.zeros(shape=(delta_sample,), dtype=np.int16) #Crea un arreglo de ceros de (32000, )
                sample[:wav.shape[0]] = wav  #Pega la parte de wav en sample
                save_sample(sample, rate, target_dir, fn, 0) #Usa la funcion save_sample para guardar el archivo. 
            # step through audio and save every delta_sample
            # discard the ending audio if it is too short
            else: #Si el archivo es mas largo que los 2 segundos
                trunc = wav.shape[0] % delta_sample #Obtiene el resto de la division de la duracion del audio por los 2 segundos que queremos
                for cnt, i in enumerate(np.arange(0, wav.shape[0]-trunc, delta_sample)):
                    start = int(i)
                    stop = int(i + delta_sample)
                    sample = wav[start:stop]
                    save_sample(sample, rate, target_dir, fn, cnt)
                    
        for fn in tqdm(os.listdir(src_dir_test)):   #Aca loopea sobre cada archivo dentro de Archivos_wav//('Charo', 'Gonza'): Es decir primero Charo1.wav, Charo2.wav
            src_fn = os.path.join(src_dir_test, fn) #Aca obtiene el directorio total del archivo que va a procesar
            rate, wav = downsampling(src_fn, args.sr)  #Aca llama a la función downsample_mono. Le pasa Archivos_wav/Charo/el wav y el sr que se quiere (16000)
            #mask, y_mean = envelope(wav, rate, threshold=args.threshold)  #Por ahora la evito
            #wav = wav[mask]
            delta_sample = int(dt*rate)   # 16000 * 2 = 32000 : Cada sample va a tener 32000 muestras. Las que queden menores se rellenan con 0.

            # cleaned audio is less than a single sample
            # pad with zeros to delta_sample size
            if wav.shape[0] < delta_sample: #Si el archivo es mas corto que los 2 segundos que queremos le agregamos ceros
                sample = np.zeros(shape=(delta_sample,), dtype=np.int16) #Crea un arreglo de ceros de (32000, )
                sample[:wav.shape[0]] = wav  #Pega la parte de wav en sample
                save_sample(sample, rate, target_dir_test, fn, 0) #Usa la funcion save_sample para guardar el archivo. 
            # step through audio and save every delta_sample
            # discard the ending audio if it is too short
            else: #Si el archivo es mas largo que los 2 segundos
                trunc = wav.shape[0] % delta_sample #Obtiene el resto de la division de la duracion del audio por los 2 segundos que queremos
                for cnt, i in enumerate(np.arange(0, wav.shape[0]-trunc, delta_sample)):
                    start = int(i)
                    stop = int(i + delta_sample)
                    sample = wav[start:stop]
                    save_sample(sample, rate, target_dir_test, fn, cnt)



100%|██████████| 1/1 [00:02<00:00,  2.00s/it]
